In [86]:
import scipy.io
import numpy as np

# Load .mat file
file_path = "./PPG_data/Compiled/PPGECG_all.mat"
mat_data = scipy.io.loadmat(file_path)

# Check keys in the .mat file
print(mat_data.keys())


dict_keys(['__header__', '__version__', '__globals__', 'S', 'None', 'fs', 'labels', 'spikes_all', '__function_workspace__'])


In [87]:
PPG_ECG = mat_data['S']
labels = mat_data['labels']
PPG = PPG_ECG[:, :, 0]
ECG = PPG_ECG[:, :, 1]
PPG = np.transpose(PPG)
ECG = np.transpose(ECG)

In [88]:
rr = mat_data['spikes_all']
rr = rr[0].tolist()  # list of array with shape (1,8)

In [89]:
def calculate_sdsd(rr_intervals):
    """Calculate SDSD from RR intervals."""
    rr_diff = np.diff(rr_intervals)  # Compute successive differences
    sdsd = np.std(rr_diff, ddof=1)  # Standard deviation (unbiased)
    return sdsd

In [90]:
sdsd_all = [] 
for i in rr:
    sdsd = calculate_sdsd(i)
    sdsd_all.append(sdsd.item())

ECG_labels = [1 if i>100 else 0 for i in sdsd_all]
ECG_labels = np.array(ECG_labels)
# ECG_labels_onehot = np.eye(2)[ECG_labels]
ECG_labels_onehot = ECG_labels

In [91]:
print(ECG_labels_onehot.shape)
print(ECG_labels_onehot[:10])

(27245,)
[0 0 0 0 0 1 0 0 1 0]


In [92]:
ectopics = np.array([0, 1])
print(ectopics)

[0 1]


In [93]:
print(len(sdsd_all))
print(len(ECG_labels_onehot))
print(ECG_labels_onehot.shape)
print(type(ECG_labels_onehot[0]))
print(sum([i == ectopics for i in ECG_labels_onehot]))

27245
27245
(27245,)
<class 'numpy.int64'>
[14464 12781]


In [94]:
from sklearn.model_selection import train_test_split

In [95]:
print(PPG.shape)
PPG = np.expand_dims(PPG, axis=1)
PPG = np.expand_dims(PPG, axis=1)

(27245, 1920)


In [96]:
x_train, x_rest, y_train, y_rest = train_test_split(PPG, ECG_labels_onehot, test_size=0.4, random_state=16, shuffle=True)

In [97]:
x_val, x_test, y_val, y_test = train_test_split(x_rest, y_rest, test_size=0.5, random_state=16, shuffle=True)

In [98]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)

(16347, 1, 1, 1920)
(16347,)
(5449, 1, 1, 1920)
(5449,)
(5449, 1, 1, 1920)
(5449,)


In [99]:
print(len(x_train))

16347


In [100]:
with open('./PPG_data/splitted_data/x_train.npy', 'wb') as f:
    np.save(f, x_train)
with open('./PPG_data/splitted_data/y_train.npy', 'wb') as f:
    np.save(f, y_train)
with open('./PPG_data/splitted_data/x_val.npy', 'wb') as f:
    np.save(f, x_val)
with open('./PPG_data/splitted_data/y_val.npy', 'wb') as f:
    np.save(f, y_val)
with open('./PPG_data/splitted_data/x_test.npy', 'wb') as f:
    np.save(f, x_test)
with open('./PPG_data/splitted_data/y_test.npy', 'wb') as f:
    np.save(f, y_test)